In [1]:
import gym
import numpy as np

env = gym.make('MountainCar-v0')
env.observation_space

Box([-1.2  -0.07], [0.6  0.07], (2,), float32)

In [2]:
class Pile_layer():
    def __init__(self, dimension, input_dim, amount, offset):
        self.dimension = dimension
        self.amount = amount
        self.offset = offset
        self.input_dim = input_dim
        self.initialize_layer()
        self.total_dimension = (dimension[0]+1)*(dimension[1]+1)*amount
    
    def initialize_layer(self):
        width = []
        self.x_ranges = []
        self.y_ranges = []
        for i in range(len(self.dimension)):
            width.append((input_dim[i][1]-input_dim[i][0])/(self.dimension[i]))
        for num in range(self.amount):
            this_x_offset = self.offset[0]*width[0]*num
            this_y_offset = self.offset[1]*width[1]*num
            this_x_ranges = []
            this_y_ranges = []
            for i in range(self.dimension[0]+1):
                if i == 0:
                    this_x_ranges.append(self.input_dim[0][0])
                    if this_x_offset != 0:
                        this_x_ranges.append(self.input_dim[0][0]+this_x_offset+width[0]*i)
                elif i == self.dimension[0]:
                    this_x_ranges.append(self.input_dim[0][1])
                    if this_x_offset == 0:
                        this_x_ranges.append(-np.inf)
                else:
                    this_x_ranges.append(self.input_dim[0][0]+this_x_offset+width[0]*i)                        
                        
            for i in range(self.dimension[1]+1):
                if i == 0:
                    this_y_ranges.append(self.input_dim[1][0])
                    if this_y_offset != 0:
                        this_y_ranges.append(self.input_dim[1][0]+this_y_offset+width[1]*i)
                elif i == self.dimension[1]:
                    this_y_ranges.append(self.input_dim[1][1])
                    if this_y_offset == 0:
                        this_y_ranges.append(-np.inf)                    
                else:
                    this_y_ranges.append(self.input_dim[1][0]+this_y_offset+width[1]*i) 
        
            self.x_ranges.append(this_x_ranges)
            self.y_ranges.append(this_y_ranges)
            
        self.x_ranges = np.asarray(self.x_ranges)
        self.y_ranges = np.asarray(self.y_ranges)
    
    def get_vector(self, state):
        state_matrix = np.zeros((self.amount, self.dimension[0]+1, self.dimension[1]+1))
        for num in range(self.amount):
            x_tile = np.argmin([np.inf if x<=0 else x for x in self.x_ranges[num]-state[0]])-1
            y_tile = np.argmin([np.inf if x<=0 else x for x in self.y_ranges[num]-state[1]])-1
            state_matrix[num, x_tile, y_tile] = 1
        
        return state_matrix.flatten()
            
            

input_dim = [[env.observation_space.low[0],env.observation_space.high[0]],[env.observation_space.low[1],env.observation_space.high[1]]]
piler = Pile_layer([15,15],input_dim, 5, [0.2,0.2])

In [3]:
env.reset()
#piler.get_vector([0,0]).reshape((5,9,9))

array([-0.5087216,  0.       ], dtype=float32)

In [4]:
#import time

actions_shared = [0,1,2]

# Define shared parameters
gamma_shared = 0.9
epsilon_shared = 0.15
num_episodes_shared = 2
num_agents_shared = 1
alpha_shared = 0.2

#actions_shared = env.get_actions()
num_actions_shared = len(actions_shared)

pi_init_shared = [1/num_actions_shared for _ in range(num_actions_shared)] #uniform distribution
q_init_shared = 0 # Initialize state-action-pairs that have not been seen yet with 0

def init_pi_q(pi, q, pi_init, q_init, state, actions, terminal):
    # function to initialize pi and q when states/state-action-pairs are visited the first time
    # In contrast to initializing the variables for each state/state-action-pair at the beginning
    # this reduces the size of pi and q and thus reduces memory requirements and reduces computation time by approx. 50%
    if state not in pi.keys():
        pi[state] = pi_init
        for action in actions:
            if terminal:
                q[(state, action)] = 0
            else:
                q[(state, action)] = q_init
    
    return pi, q

def init_pi_q_e(pi, q, e, pi_init, q_init, state, actions, terminal):
    # function to initialize pi and q when states/state-action-pairs are visited the first time
    # In contrast to initializing the variables for each state/state-action-pair at the beginning
    # this reduces the size of pi and q and thus reduces memory requirements and reduces computation time by approx. 50%
    if state not in pi.keys():
        pi[state] = pi_init
        for action in actions:
            e[(state, action)] = 0            
            if terminal:
                q[(state, action)] = 0
            else:
                q[(state, action)] = q_init
    
    return pi, q, e

In [5]:
alpha = 0.1
gamma = 0.95
epsilon_start = 0.15
num_episodes = 1000
num_agents = 1
pi_init = pi_init_shared
q_init = q_init_shared

actions = actions_shared
num_actions = len(actions)

sarsa_rewards = []
agent_id = 0

pi_init = [1/num_actions for _ in range(num_actions)]
q_init = 0

while agent_id < num_agents:
    agent_id += 1

    # Initialize policy "pi", state-action-pair-values "q" and dict of returns
    # Dictionaries will be filled during training, Epsilon-soft pi guarantees that each state will be visited (for infinite episodes)
    pi = {}
    sap = {}
    q = {}
    returns = {}
    theta = np.full((3, piler.total_dimension), 1, dtype=float)

    episode_id = 0
    this_agent_rewards = []
    #this_agent_q = np.zeros((3, piler.total_dimension))
    #counter = 0
    finished = False
    while episode_id < num_episodes:
        episode_id += 1
        epsilon = epsilon_start-epsilon_start/(num_episodes)*episode_id
        #if episode_id%10==0:
        #    print(episode_id)
        this_episode_sap = []
        this_episode_rewards = []
        this_state = env.reset()
        #print(this_state)
        this_state = piler.get_vector(this_state)
        #this_state_hash = hash(str(this_state))
        #print(this_state_hash)
        terminal = False
        #pi, q = init_pi_q(pi, q, pi_init, q_init, this_state_hash, actions, terminal)
        q_all_a = np.array([np.sum(theta[x]*this_state) for x in actions])
        a_star = np.random.choice(np.flatnonzero(q_all_a == q_all_a.max()))
        #pi[this_state_hash] = [epsilon/num_actions if x!=a_star else 1-epsilon+epsilon/num_actions for x in actions]
        this_action = np.random.choice(actions, p=[epsilon/num_actions if x!=a_star else 1-epsilon+epsilon/num_actions for x in actions])
        #print(theta[this_action].shape,this_state.shape)
        this_q = np.sum(theta[this_action]*this_state)
           
        #print(q_all_a)
        #print(pi[this_state_hash])
        #(next_state, reward, terminal, _) = env.step(this_action)
        #print(next_state)
        #print(reward)
        #print(terminal)
        counter=0
        while True:
            counter += 1
            (next_state, reward, terminal, _) = env.step(this_action)
            #if terminal and counter<200:
            #    print(episode_id, 'Finished')
            #print(next_state[0])
            #if next_state[0]<-1.2: 
            #    this_state = env.reset()
            #    #print(this_state)
            #    this_state = piler.get_vector(this_state)
            #    this_state_hash = hash(str(this_state))
            #    #print(this_state_hash)
            #    terminal = False
            #    #pi, q = init_pi_q(pi, q, pi_init, q_init, this_state_hash, actions, terminal)
            #    q_all_a = np.array([np.sum(theta[x]*this_state) for x in actions])
            #    a_star = np.random.choice(np.flatnonzero(q_all_a == q_all_a.max()))
            #    #pi[this_state_hash] = [epsilon/num_actions if x!=a_star else 1-epsilon+epsilon/num_actions for x in actions]
            #    this_action = np.random.choice(actions, p=[epsilon/num_actions if x!=a_star else 1-epsilon+epsilon/num_actions for x in actions])
            #    #print(theta[this_action].shape,this_state.shape)
            #    this_q = np.sum(theta[this_action]*this_state)
            #if next_state[0]>0.6:
            #    reward = 10
            #    terminal = True
            #print(next_state)
            next_state = piler.get_vector(next_state)
            #next_state_hash = hash(str(next_state))
            #pi, q = init_pi_q(pi, q, pi_init, q_init, next_state_hash, actions, terminal)
            q_all_a = np.array([np.sum(theta[x]*next_state) for x in actions])
            a_star = np.random.choice(np.flatnonzero(q_all_a == q_all_a.max()))
            #pi[next_state_hash] = [epsilon/num_actions if x!=a_star else 1-epsilon+epsilon/num_actions for x in actions]
            next_action = np.random.choice(actions, p=[epsilon/num_actions if x!=a_star else 1-epsilon+epsilon/num_actions for x in actions])
            next_q = np.sum(theta[next_action]*next_state)
            #print(next_action)
            #print(np.sum(theta[this_action]*this_state))
            theta[this_action] = theta[this_action] +alpha * (reward+gamma*next_q-this_q)*this_state
            #print(np.sum(theta[this_action]*this_state))
            #print()
            #q[(this_state, this_action)] = q[(this_state, this_action)] + alpha*(reward+gamma*q[(next_state, next_action)] - q[(this_state, this_action)])

            this_state = next_state
            #this_state_hash = next_state_hash
            this_action = next_action
            this_q = next_q

            this_episode_rewards.append(reward)
            #counter += 1
            #if finished:
            #    env.render()
            #if counter%10000 == 0:
            #    print(theta.reshape((3,3,9,9))[:,0,:,:])
            #    print()
            #    print()
            #if episode_id%200 == 0:
            #    #print(episode_id)
            #    env.render()
            if terminal:
                break

        this_agent_rewards.append(sum(this_episode_rewards))
    sarsa_rewards.append(this_agent_rewards)

#print(np.max(theta))
env.close()

In [6]:
#from matplotlib import pyplot as plt
#print(theta[0][:121])
print(np.sum([1 for x in sarsa_rewards[0] if x >-200 ]))
print(np.sum([1 for x in sarsa_rewards[0] if x ==-200 ]))
#print(np.sum(theta[2]*piler.get_vector([-0.1,-0.01])))

770
230


In [ ]:
import pdb
#epsilon = 0
episode_id=0
while episode_id < 100:
    episode_id += 1
    if episode_id%10==0:
        print(episode_id)
    this_episode_sap = []
    this_episode_rewards = []
    this_state = env.reset()
    #print(this_state)
    this_state = piler.get_vector(this_state)
    this_state_hash = hash(str(this_state))
    #print(this_state_hash)
    terminal = False
    pi, q = init_pi_q(pi, q, pi_init, q_init, this_state_hash, actions, terminal)
    q_all_a = np.array([np.sum(theta[x]*this_state) for x in actions])
    a_star = np.random.choice(np.flatnonzero(q_all_a == q_all_a.max()))
    pi[this_state_hash] = [epsilon/num_actions if x!=a_star else 1-epsilon+epsilon/num_actions for x in actions]
    this_action = np.random.choice(actions, p=pi[this_state_hash])
    #print(theta[this_action].shape,this_state.shape)
    this_q = np.sum(theta[this_action]*this_state)

    #print(q_all_a)
    #print(pi[this_state_hash])
    #(next_state, reward, terminal, _) = env.step(this_action)
    #print(next_state)
    #print(reward)
    #print(terminal)
    counter = 0
    while True:
        (next_state, reward, terminal, _) = env.step(this_action)
        #print(next_state)
        next_state = piler.get_vector(next_state)
        next_state_hash = hash(str(next_state))
        pi, q = init_pi_q(pi, q, pi_init, q_init, next_state_hash, actions, terminal)
        q_all_a = np.array([np.sum(theta[x]*next_state) for x in actions])
        a_star = np.random.choice(np.flatnonzero(q_all_a == q_all_a.max()))
        pi[next_state_hash] = [epsilon/num_actions if x!=a_star else 1-epsilon+epsilon/num_actions for x in actions]
        next_action = np.random.choice(actions, p=pi[next_state_hash])
        next_q = np.sum(theta[next_action]*next_state)
        #print(next_action)
        #print(np.sum(theta[this_action]*this_state))
        #theta[this_action] = theta[this_action] +alpha * (reward+gamma*next_q-this_q)*this_state
        #print(np.sum(theta[this_action]*this_state))
        #print()
        #q[(this_state, this_action)] = q[(this_state, this_action)] + alpha*(reward+gamma*q[(next_state, next_action)] - q[(this_state, this_action)])

        this_state = next_state
        this_state_hash = next_state_hash
        this_action = next_action
        this_q = next_q

        this_episode_rewards.append(reward)
        counter += 1
        env.render()
        #if episode_id%100 == 0:
            #env.render()
        if terminal:
            #print(counter)
            break

env.close()

10
20


In [22]:
print(np.sum(theta[0]*next_state))
print(np.sum(theta[1]*next_state))
print(np.sum(theta[2]*next_state))
env.step(2)

-10.000013135092747
-10.000016976805657
-10.000001179422672


(array([-5.2924824e-01,  1.4771317e-04], dtype=float32), -1.0, False, {})

In [ ]:
def sarsa_control(env, piler, alpha, gamma, epsilon, pi_init, q_init, num_episodes, num_agents):
    actions = actions_shared
    num_actions = len(actions)
    
    sarsa_rewards = []
    agent_id = 0
    
    pi_init = [1/num_actions for _ in range(num_actions)]
    q_init = 0

    while agent_id < num_agents:
        agent_id += 1
        
        # Initialize policy "pi", state-action-pair-values "q" and dict of returns
        # Dictionaries will be filled during training, Epsilon-soft pi guarantees that each state will be visited (for infinite episodes)
        pi = {}
        sap = {}
        q = {}
        returns = {}
        theta = np.zeros((len(actions), piler.amount, piler.dimension[0]+1, piler.dimension[1]+1))
        
        episode_id = 0
        this_agent_rewards = []

        while episode_id < num_episodes:
            episode_id += 1
            
            this_episode_sap = []
            this_episode_rewards = []
            this_state = env.reset()
            terminal = False
            pi, q = init_pi_q(pi, q, pi_init, q_init, this_state, actions, terminal)
            q_all_a = np.array([q[(this_state,x)] for x in actions])
            a_star = np.random.choice(np.flatnonzero(q_all_a == q_all_a.max()))
            pi[this_state] = [epsilon/num_actions if x!=a_star else 1-epsilon+epsilon/num_actions for x in actions]
            this_action = np.random.choice(actions, p=pi[this_state])
            while True:
                next_state, reward, terminal = env.step(this_action)
                pi, q = init_pi_q(pi, q, pi_init, q_init, next_state, actions, terminal)
                q_all_a = np.array([q[(next_state,x)] for x in actions])
                a_star = np.random.choice(np.flatnonzero(q_all_a == q_all_a.max()))
                pi[next_state] = [epsilon/num_actions if x!=a_star else 1-epsilon+epsilon/num_actions for x in actions]
                next_action = np.random.choice(actions, p=pi[next_state])
                
                q[(this_state, this_action)] = q[(this_state, this_action)] + alpha*(reward+gamma*q[(next_state, next_action)] - q[(this_state, this_action)])
                
                this_state = next_state
                this_action = next_action
                
                this_episode_rewards.append(reward)
                
                if terminal:
                    break
            
            this_agent_rewards.append(sum(this_episode_rewards))
        sarsa_rewards.append(this_agent_rewards)
    
    return sarsa_rewards

alpha = alpha_shared
gamma = gamma_shared
epsilon = epsilon_shared
num_episodes = num_episodes_shared
num_agents = num_agents_shared
pi_init = pi_init_shared
q_init = q_init_shared

sarsa_rewards = sarsa_control(env, piler, alpha, gamma, epsilon, pi_init, q_init, num_episodes, num_agents)